1. Configurar entorno

In [1]:
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install jellyfish
import jellyfish

2. Tratamiento de datos. "Limpiar"

In [18]:
def tratamientoTexto(texto):
 trans = str.maketrans('áéíóú','aeiou')
 texto = texto.lower()
 #print('Minúsculas: ',texto)
 texto = texto.translate(trans)
 #print('Acentos: ',texto)
 texto = re.sub(r"[^\w\s]", '', texto)
 #print('Símbolos: ',texto)
 texto = " ".join(texto.split())
 return texto

3. Cargar bases de datos


In [32]:
# Importando bases de dialogo fluído
txtFolderPath = '/content'
listaDocs=[x for x in os.listdir(txtFolderPath) if x.endswith(".txt")]
listaDialogos, listaRespuestas, listaTipoDialogo = [],[],[]
for idx in range(len(listaDocs)):
 f=open(txtFolderPath+'/'+listaDocs[idx], 'r', encoding='utf-8', errors='ignore')
 flag,posicion = True,0
 for line in f.read().split('\n'):
  if flag:
    line = tratamientoTexto(line)
    listaDialogos.append(line) # clean it
    listaTipoDialogo.append(listaDocs[idx][:2])
  else:
    listaRespuestas.append(line)
    posicion +=1
  flag = not flag
# Creando Dataframe de diálogos
datos = {'dialogo':listaDialogos,'respuesta':listaRespuestas,'tipo':listaTipoDialogo,'interseccion':0,'similarity':0,'jaro_winkler':0,'probabilidad':0}
dfDialogos = pd.DataFrame(datos)
dfDialogos = dfDialogos.astype({ 'interseccion': 'float', 'similarity': 'float', 'jaro_winkler': 'float', 'probabilidad': 'float' })

# drop.duplicate function
dfDialogos.drop_duplicates(keep='first')
# RESET INDEX
dfDialogos.reset_index(drop=True, inplace= True)

4. Buscar la respuesta del Chatbot


In [20]:
vectorizer = TfidfVectorizer()
dialogosNum = vectorizer.fit_transform(dfDialogos['dialogo'])


#Función que comprueba si el usuario inició un diálogo
def dialogo(user_response):
 df = dfDialogos.copy()
 respuestaNum = vectorizer.transform([user_response])
 for idx,row in df.iterrows():
   df.at[idx,'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
   df.at[idx,'similarity'] = cosine_similarity(dialogosNum[idx], respuestaNum)[0][0]
   df.at[idx,'jaro_winkler'] = jellyfish.jaro_winkler_similarity(user_response,row['dialogo'])
   df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
 df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
 return df.head(5)


5. Ejecutar el Chatbot

In [36]:
#pregunta = 'Hola, ¿quién eres?'
#pregunta = 'Te comprarias un iPhone 16'
#pregunta = 'Pasa pues?'
pregunta = 'Necesito algo mas de inforamcion'
user_response = tratamientoTexto(pregunta)
print(user_response)
respuesta = dialogo(user_response)
respuesta

necesito algo mas de inforamcion


,dialogo,respuesta,tipo,interseccion,similarity,jaro_winkler,probabilidad
6,esa no es la informacion que necesito,"Lo siento, entendí mal. Ya actualicé mi sistem...",ER,0.2,0.383159,0.709678,0.709678
110,hola como te identificas,"Hola, me identifico como Pat, ¿y tú cómo te ll...",ID,0.0,0.000000,0.700893,0.700893
114,hola como te identificas,"Hola, me identifico como Pat, ¿y tú cómo te ll...",ID,0.0,0.000000,0.700893,0.700893
217,muchas gracias por la informacion,"De nada, estoy aquí para ayudar en todo lo que...",GR,0.0,0.000000,0.699609,0.699609
166,buenos dias como amaneciste,"Muy bien, gracias. ¿En qué puedo ayudarte hoy?",SA,0.0,0.000000,0.697741,0.697741


6. Comprobaciones

In [27]:
# TEST DE TRATAMIENTO DE TEXTO
texto1 = 'Hola, Qué tal?'
print(tratamientoTexto(texto1))

hola que tal


In [28]:
#
dfDialogos

,dialogo,respuesta,tipo,interseccion,similarity,jaro_winkler,probabilidad
0,mentira,"Lo siento, entendí mal. Ya actualicé mi sistem...",ER,0,0,0,0
1,errado,"Lo siento, entendí mal. Ya actualicé mi sistem...",ER,0,0,0,0
2,equivocado,"Lo siento, entendí mal. Ya actualicé mi sistem...",ER,0,0,0,0
3,eso no es lo que pregunte,"Lo siento, entendí mal. Ya actualicé mi sistem...",ER,0,0,0,0
4,eso no tiene sentido,Me disculpo por la confusión. Ya actualicé mi ...,ER,0,0,0,0
...,...,...,...,...,...,...,...
233,gracias por tu eficiencia,"No hay de qué, siempre estoy aquí para brindar...",GR,0,0,0,0
234,gracias por tu atencion al detalle,Me alegra que te haya sido útil mi atención al...,GR,0,0,0,0
235,gracias por tu pronta respuesta,"No hay problema, siempre estoy aquí para respo...",GR,0,0,0,0
236,gracias por tu orientacion,"No hay de qué, siempre estoy aquí para orienta...",GR,0,0,0,0


In [29]:
# METODO INTERSECCION
texto1 = 'Quien eres tu'
texto2 = 'hola quien eres'
print(len(set(texto1.split()) & set(texto2.split()))/len(texto1.split()))

0.3333333333333333


In [30]:
# similar del coseno
texto1 = 'Quien eres tu'
texto2 = 'hola quien eres'
texto1 = vectorizer.transform([texto1])
texto2 = vectorizer.transform([texto2])
print(cosine_similarity(texto1, texto2)[0][0])

0.8066711810398435


In [31]:
#Método Jaro-Winkler
texto1 = 'quien eres tu'
texto2 = 'hola quien eres'
print(jellyfish.jaro_winkler_similarity(texto1,texto2))

0.7083139083139084
